In [1]:
from Modules import *
import symengine as se
import sympy

se.init_printing(pretty_print=True)
sympy.init_printing(use_latex='mathjax')

In [2]:
iG = se.Symbol("i_G" , real = True)
c = se.Symbol("c" , real = True)
s = se.Symbol("s" , real = True)
m = se.Symbol("m" , real = True)
CA = se.Symbol("C_A" , real = True)
CM = se.Symbol("C_M" , real = True)
km = se.Symbol("k_m" , real = True)

params = [iG,c,s,m,CA,CM,km]

In [3]:
# q_var = DynamicSymbols(["q_M","q_A"],1,2,True)
q_var = DynamicSymbols("q",2,2)

u_var = DynamicSymbols("M_{Mot}",1,0)

u = u_var.vars
[q,q_dot,q_ddot] = q_var.vars
[[qM,qA],[qM_dot, qA_dot],[qM_ddot, qA_ddot]] = q_var.vars

display(q_var, u_var)
M_Mot = u[0]

In [4]:
K_J_AS = se.diag(0,0,CA)
K_J_AS

[0, 0, 0]
[0, 0, 0]
[0, 0, C_A]

In [5]:
K_J_MS = se.zeros(3)
K_J_MS[2,2] = CM


K_J_MS

[0, 0, 0]
[0, 0, 0]
[0, 0, C_M]

In [6]:
R_IK = Drehmatrix(se.Matrix([0,0,qM]))
display(R_IK)

[cos(q_0(t)), -sin(q_0(t)), 0]
[sin(q_0(t)), cos(q_0(t)), 0]
[0, 0, 1]

In [7]:
K_r_s = se.Matrix([s,0,0])
K_r_s

[s]
[0]
[0]

In [8]:
K_w_K = se.Matrix([0,0,qA_dot+qM_dot])
K_w_K

[0]
[0]
[\dot{q}_0(t) + \dot{q}_1(t)]

In [9]:
K_w_M = se.Matrix([0,0,iG*qM_dot])
K_w_M

[0]
[0]
[i_G*\dot{q}_0(t)]

In [10]:
K_v_S = diff_t(K_r_s) + K_w_K.cross(K_r_s)
K_v_S

[0]
[s*(\dot{q}_0(t) + \dot{q}_1(t))]
[0]

In [11]:
K_p = m * K_v_S
K_p

[0]
[m*s*(\dot{q}_0(t) + \dot{q}_1(t))]
[0]

In [12]:
K_L_A = K_J_AS * K_w_K
K_L_A

[0]
[0]
[(\dot{q}_0(t) + \dot{q}_1(t))*C_A]

In [13]:
K_L_M = K_J_MS * K_w_M
K_L_M

[0]
[0]
[i_G*C_M*\dot{q}_0(t)]

In [14]:
K_Me_A = se.Matrix([0,0,-qA * c])
K_Me_A

[0]
[0]
[-c*q_1(t)]

In [15]:
K_Me_M = se.Matrix([0,0,M_Mot]) - K_Me_A / iG
K_Me_M

[0]
[0]
[c*q_1(t)/i_G + M_{Mot}(t)]

In [16]:
display(diff_t(K_p), K_p)
part1 = K_v_S.jacobian(q_dot).transpose() * (diff_t(K_p) + K_w_K.cross(K_p))


part1

[0]
[m*s*(\ddot{q}_0(t) + \ddot{q}_1(t))]
[0]

[0]
[m*s*(\dot{q}_0(t) + \dot{q}_1(t))]
[0]

[m*s**2*(\ddot{q}_0(t) + \ddot{q}_1(t))]
[m*s**2*(\ddot{q}_0(t) + \ddot{q}_1(t))]

In [17]:
part2 = K_w_K.jacobian(q_dot).transpose() * (diff_t(K_L_A) + K_w_K.cross(K_L_A) - K_Me_A)
part2

[c*q_1(t) + (\ddot{q}_0(t) + \ddot{q}_1(t))*C_A]
[c*q_1(t) + (\ddot{q}_0(t) + \ddot{q}_1(t))*C_A]

In [18]:
part3 = K_w_M.jacobian(q_dot).transpose() * (diff_t(K_L_M) + K_w_M.cross(K_L_M) - K_Me_M)
part3

[i_G*(i_G*C_M*\ddot{q}_0(t) - (c*q_1(t)/i_G + M_{Mot}(t)))]
[0]

In [19]:
parts = part1 + part2 + part3
parts

[c*q_1(t) + i_G*(i_G*C_M*\ddot{q}_0(t) - (c*q_1(t)/i_G + M_{Mot}(t))) + (\ddot{q}_0(t) + \ddot{q}_1(t))*C_A + m*s**2*(\ddot{q}_0(t) + \ddot{q}_1(t))]
[c*q_1(t) + (\ddot{q}_0(t) + \ddot{q}_1(t))*C_A + m*s**2*(\ddot{q}_0(t) + \ddot{q}_1(t))]

In [20]:
M = parts.jacobian(q_ddot)
M

[C_A + i_G**2*C_M + m*s**2, C_A + m*s**2]
[C_A + m*s**2, C_A + m*s**2]

In [21]:
g = parts - M @  q_ddot
g = g.subs(dict(zip(q_ddot, [0,0]))).expand()
g

[-i_G*M_{Mot}(t)]
[c*q_1(t)]

In [22]:
f = q_dot.col_join(-M.inv() * g)
display(se.cse(f))
f

([(x0, C_A + m*s**2),
  (x1, (x0 + i_G**2*C_M)**(-1)),
  (x2, x0**2*x1),
  (x3, (x0 - x2)**(-1)),
  (x4, x1*i_G*M_{Mot}(t)),
  (x5, c*x3*q_1(t))],
 [\dot{q}_0(t), \dot{q}_1(t), x4*(1 + x2*x3) + x0*x1*x5, -x5 - x0*x4*x3])

[\dot{q}_0(t)]
[\dot{q}_1(t)]
[i_G*(1 + (C_A + m*s**2)**2/((C_A + i_G**2*C_M + m*s**2)*(C_A + m*s**2 - (C_A + m*s**2)**2/(C_A + i_G**2*C_M + m*s**2))))*M_{Mot}(t)/(C_A + i_G**2*C_M + m*s**2) + c*q_1(t)*(C_A + m*s**2)/((C_A + i_G**2*C_M + m*s**2)*(C_A + m*s**2 - (C_A + m*s**2)**2/(C_A + i_G**2*C_M + m*s**2)))]
[-c*q_1(t)/(C_A + m*s**2 - (C_A + m*s**2)**2/(C_A + i_G**2*C_M + m*s**2)) - i_G*(C_A + m*s**2)*M_{Mot}(t)/((C_A + i_G**2*C_M + m*s**2)*(C_A + m*s**2 - (C_A + m*s**2)**2/(C_A + i_G**2*C_M + m*s**2)))]

In [23]:
h = q.col_join(q_dot)
dynSys = DynamicSystem(h,u)
dynSys.addStateEquations(f)
dynSys.addOutput(h,"x")
dynSys.addInput(u,"u")
dynSys.addParameter(params)
display(dynSys.x_dot)

#sys = Robotik_System(f,h,q_var,u_var,params,False)

[\dot{q}_0(t)]
[\dot{q}_1(t)]
[\ddot{q}_0(t)]
[\ddot{q}_1(t)]

In [24]:
[A,B,C,D] = dynSys.linearize(se.Matrix([0,0,0,0]),se.Matrix([0]))
display(A,B,C,D)

[0, 0, 1, 0]
[0, 0, 0, 1]
[0, c*(C_A + m*s**2)/((C_A + i_G**2*C_M + m*s**2)*(C_A + m*s**2 - (C_A + m*s**2)**2/(C_A + i_G**2*C_M + m*s**2))), 0, 0]
[0, -c/(C_A + m*s**2 - (C_A + m*s**2)**2/(C_A + i_G**2*C_M + m*s**2)), 0, 0]

[0]
[0]
[i_G*(1 + (C_A + m*s**2)**2/((C_A + i_G**2*C_M + m*s**2)*(C_A + m*s**2 - (C_A + m*s**2)**2/(C_A + i_G**2*C_M + m*s**2))))/(C_A + i_G**2*C_M + m*s**2)]
[-i_G*(C_A + m*s**2)/((C_A + i_G**2*C_M + m*s**2)*(C_A + m*s**2 - (C_A + m*s**2)**2/(C_A + i_G**2*C_M + m*s**2)))]

[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]

[0]
[0]
[0]
[0]

In [25]:
dynSys.write_ABCD_to_File("ss_matrices")
dynSys.write_init_File("init_params")
dynSys.write_SFunction("sfun_test")

In [26]:
display(dynSys.x.subs(DynamicSymbols._Symbol_to_printable_dict))
#sys.write_s_funs("Test")
#sys.write_matlab_funs("Test")
#display(q_var.syms)

[q0]
[q1]
[q0dot]
[q1dot]